# 19AIE312 - Deep Learning for Signal & Image Processing
## Presentation 1 Evaluation 4

### Team Members :
* Rajaneshwar G   : CB.EN.U4AIE19027
* Gnanadeep M     : CB.EN.U4AIE19047
* Sabarishwaran G : CB.EN.U4AIE19053
* Sarathi T.S     : CB.EN.U4AIE19058

### Q3

####  Classify the CIFAR10 dataset using Alexnet and Mobilenet architecture. Compare the performance of these two architectures in terms of Precision, Recall and F1 score. Plot the training and validation accuracy. Print the confusion matrix. 

# Importing required packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
 
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras
from keras.layers import Dropout, MaxPool2D, Dense, Conv2D, Flatten, BatchNormalization,GlobalAveragePooling2D
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.applications.mobilenet import MobileNet
from sklearn.metrics import confusion_matrix,classification_report

from skimage.transform import resize
import cv2

from tqdm import tqdm
import gc

# Loading data

In [ ]:
alexnet_img_size = (227,227,3)
mobilenet_img_size = (224,224,3)

In [ ]:
(x_train,y_train),(x_test,y_test) = keras.datasets.cifar10.load_data()

In [ ]:
no_of_images = 2000

In [ ]:
x_train = x_train[:no_of_images]
y_train = y_train[:no_of_images]
x_test = x_test[:int(no_of_images/2)]
y_test = y_test[:int(no_of_images/2)]

In [ ]:
Y_test = [i[0] for i in y_test]

In [ ]:
# Shape of Dataset
test_shape = np.shape(x_test)
train_shape = np.shape(x_train)

print(f'Shape of Test Data : {test_shape}')
print(f'Shape of Train Data : {train_shape}')

# Data visualization

## Sample image from train data ( before preprocessing )

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
n = np.random.randint(0,train_shape[0]) # Generating Random Number
sample = x_train[n]

plt.imshow(sample)
plt.title(str(class_names[int(y_train[n])]))

# Preprocessing

In [ ]:
def resize_images(images,img_size):
    resized_images = []
    
    for i in tqdm(range(len(images))):
        resized_images.append(resize(images[i],(img_size),mode = 'constant'))
    
    return np.array(resized_images,dtype='float32')

## Loading Data for AlexNet

In [ ]:
print('Loading train data for AlexNet :')
alex_train = resize_images(x_train,alexnet_img_size)

print('Loading test data for AlexNet :')
alex_test = resize_images(x_test,alexnet_img_size)

In [ ]:
alex_train, alex_train_labels = shuffle(alex_train, y_train, random_state=10)

In [ ]:
alex_val, alex_val_labels = alex_train[:int(no_of_images/5)], alex_train_labels[:int(no_of_images/5)]

## Loading Data for MobileNet

In [ ]:
print('Loading train data for MobileNet :')
mobile_train = resize_images(x_train,mobilenet_img_size)

print('Loading test data for MobileNet :')
mobile_test = resize_images(x_test,mobilenet_img_size)

In [ ]:
mobile_train, mobile_train_labels = shuffle(mobile_train, y_train, random_state=10)

In [ ]:
mobile_val, mobile_val_labels = mobile_train[:int(no_of_images/5)], mobile_train_labels[:int(no_of_images/5)]

## Sample image from train data ( After preprocessing )

In [ ]:
alex_sample = alex_train[n]
mobile_sample = mobile_train[n]

plt.subplot(1,2,1)
plt.imshow(alex_sample)
plt.title(str(class_names[int(alex_train_labels[n])]))

plt.subplot(1,2,2)
plt.imshow(mobile_sample)
plt.title(str(class_names[int(mobile_train_labels[n])]))

# Defining Model

## AlexNet Model

In [ ]:
AlexNet = Sequential()

AlexNet.add(Conv2D(filters = 96, kernel_size = (11,11), strides = (4,4), activation = 'relu', input_shape = (alexnet_img_size)))
AlexNet.add(BatchNormalization())
AlexNet.add(MaxPool2D(pool_size = (3,3), strides = (2,2)))

AlexNet.add(Conv2D(filters = 256, kernel_size = (5,5), strides = (1,1), activation = 'relu', padding = 'same'))
AlexNet.add(BatchNormalization())
AlexNet.add(MaxPool2D(pool_size = (3,3), strides = (2,2)))

AlexNet.add(Conv2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = 'relu', padding = 'same'))
AlexNet.add(BatchNormalization())

AlexNet.add(Conv2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = 'relu', padding = 'same'))
AlexNet.add(BatchNormalization())

AlexNet.add(Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), activation = 'relu', padding = 'same'))
AlexNet.add(BatchNormalization())
AlexNet.add(MaxPool2D(pool_size = (3,3), strides = (2,2)))

AlexNet.add(Flatten())

AlexNet.add(Dense(units = 4096, activation = 'relu'))
AlexNet.add(Dropout(0.5))

AlexNet.add(Dense(units = 4096, activation = 'relu'))
AlexNet.add(Dropout(0.5))

AlexNet.add(Dense(units = 10, activation = 'softmax'))

early_stopping = EarlyStopping(min_delta = 0.001,patience = 20,restore_best_weights = True,verbose = 0)

# Compile
AlexNet.compile(optimizer = "adam" , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])

# Train
Alex = AlexNet.fit(alex_train, alex_train_labels, batch_size = 256, epochs = 50,callbacks = [early_stopping],validation_data=(alex_val, alex_val_labels))

AlexNet.summary()

In [ ]:
AlexNet_predictions = AlexNet.predict(alex_test)
AlexNet_predictions = np.argmax(AlexNet_predictions,axis=1)

## MobileNet Model

In [ ]:
base_model = MobileNet(include_top=False, weights='imagenet',input_shape = mobilenet_img_size, classes=10)
base_model.trainable = True 

for layer in base_model.layers[:50]:
    layer.trainable =  False

MobileNet_model = Sequential()
MobileNet_model.add(base_model)
MobileNet_model.add(Flatten())
MobileNet_model.add(Dense(10,activation=('softmax')))

early_stopping = EarlyStopping(min_delta = 0.001,patience = 20,restore_best_weights = True,verbose = 0)

# Compile
MobileNet_model.compile(optimizer = "adam" , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])

# Train
Mobile = MobileNet_model.fit(mobile_train, mobile_train_labels, batch_size = 256, epochs = 50,callbacks = [early_stopping], validation_data = (mobile_val, mobile_val_labels))

MobileNet_model.summary()

In [ ]:
MobileNet_predictions = MobileNet_model.predict(mobile_test)
MobileNet_predictions = np.argmax(MobileNet_predictions,axis=1)

# AlexNet vs MobileNet : Comparing the performance

## Predicted image

In [ ]:
n = np.random.randint(0,test_shape[0]) # Generating Random Number
alex_sample = alex_test[n]
mobile_sample = mobile_test[n]

plt.subplot(1,2,1)
plt.imshow(alex_sample)
plt.title('AlexNet Prediction \n : ' + str(class_names[int(AlexNet_predictions[n])]))

plt.subplot(1,2,2)
plt.imshow(mobile_sample)
plt.title('MobileNet Prediction \n : ' + str(class_names[int(MobileNet_predictions[n])]))

## Plotting the confusion matrix 

In [ ]:
alex_cm = confusion_matrix(Y_test, AlexNet_predictions)
mobile_cm = confusion_matrix(Y_test, MobileNet_predictions)

plt.figure(figsize = (30,10))

plt.subplot(1,2,1)
sns.heatmap(alex_cm,cmap = 'Blues',annot = True, xticklabels = class_names, yticklabels = class_names)

plt.subplot(1,2,2)
sns.heatmap(mobile_cm,cmap = 'Blues',annot = True, xticklabels = class_names, yticklabels = class_names)

## Classification Report ( Precision , recall , F1 Score ) 

In [ ]:
def precision(cm):
    p = {}
    
    for i in range(len(cm)):
        p[i] = cm[i,i] / sum(cm[:,i])
    
    return p

In [ ]:
def recall(cm):
    r = {}
    
    for i in range(len(cm)):
        r[i] = cm[i,i]/sum(cm[i,:])
    
    return r

In [ ]:
def f1_score(cm):
    f1 = {}
    
    for i in range(len(cm)):
        f1[i] = 2 * (cm[i,i] / sum(cm[:,i])) * (cm[i,i]/sum(cm[i,:])) / ((cm[i,i] / sum(cm[:,i])) + (cm[i,i]/sum(cm[i,:])))
    
    return f1

In [ ]:
alex_p = precision(alex_cm)
alex_r = recall(alex_cm)
alex_f1 = f1_score(alex_cm)

In [ ]:
mobile_p = precision(mobile_cm)
mobile_r = recall(mobile_cm)
mobile_f1 = f1_score(mobile_cm)

In [ ]:
Precision = {
    'Alex Precision' : alex_p,
    'Mobile Precision' : mobile_p
}

Precision = pd.DataFrame(Precision)
Precision

In [ ]:
Recall = {
    'Alex Recall' : alex_r,
    'Mobile Recall' : mobile_r
}

Recall = pd.DataFrame(Recall)
Recall

In [ ]:
F1_score = {
    'Alex F1 Score' : alex_f1,
    'Mobile F1 Score' : mobile_f1
}

F1_score = pd.DataFrame(F1_score)
F1_score

## Training & Validation Accuracy

In [ ]:
plt.figure(figsize = (30,10))

plt.subplot(1,2,1)
alex_train_acc = Alex.history['accuracy']
alex_val_acc = Alex.history['val_accuracy']
alex_epoch = [i for i in range(len(alex_val_acc))]
plt.plot(alex_epoch , alex_train_acc , 'go-' , label = 'Training Accuracy')
plt.plot(alex_epoch, alex_val_acc , 'ro-' , label = 'Validation Accuracy')
plt.title('Training & Validation Accuracy for AlexNet')
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")

plt.subplot(1,2,2)
mobile_train_acc = Mobile.history['accuracy']
mobile_val_acc = Mobile.history['val_accuracy']
mobile_epoch = [i for i in range(len(mobile_val_acc))]
plt.plot(mobile_epoch , mobile_train_acc , 'go-' , label = 'Training Accuracy')
plt.plot(mobile_epoch , mobile_val_acc , 'ro-' , label = 'Validation Accuracy')
plt.title('Training & Validation Accuracy for MobileNet')
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")